In [1]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import chi2_contingency, ttest_ind

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


In [8]:
# cargamos el dataframe correspondiente 
df = pd.read_csv("archivos/Amazon Sales FY2020-21.csv")

display(df.head(10))

,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,category,payment_method,bi_st,cust_id,year,month,ref_num,Name Prefix,First Name,Middle Initial,Last Name,Gender,age,full_name,E Mail,Sign in date,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent
0,100354678,01/10/2020,received,574772,oasis_Oasis-064-36,21,89.9,1798.0,0.0,1798.0,Men's Fashion,cod,Valid,60124,2020,Oct-20,987867,Drs.,Jani,W,Titus,F,43,"Titus, Jani",jani.titus@gmail.com,8/22/2006,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
1,100354678,01/10/2020,received,574774,Fantastic_FT-48,11,19.0,190.0,0.0,190.0,Men's Fashion,cod,Valid,60124,2020,Oct-20,987867,Drs.,Jani,W,Titus,F,43,"Titus, Jani",jani.titus@gmail.com,8/22/2006,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
2,100354680,01/10/2020,complete,574777,mdeal_DMC-610-8,9,149.9,1199.2,0.0,1199.2,Men's Fashion,cod,Net,60124,2020,Oct-20,987867,Drs.,Jani,W,Titus,F,43,"Titus, Jani",jani.titus@gmail.com,8/22/2006,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
3,100354680,01/10/2020,complete,574779,oasis_Oasis-061-36,9,79.9,639.2,0.0,639.2,Men's Fashion,cod,Net,60124,2020,Oct-20,987867,Drs.,Jani,W,Titus,F,43,"Titus, Jani",jani.titus@gmail.com,8/22/2006,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
4,100367357,13/11/2020,received,595185,MEFNAR59C38B6CA08CD,2,99.9,99.9,0.0,99.9,Men's Fashion,cod,Valid,60124,2020,Nov-20,987867,Drs.,Jani,W,Titus,F,43,"Titus, Jani",jani.titus@gmail.com,8/22/2006,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
5,100367357,13/11/2020,received,595186,MEFBUY59B7C3DDC2CA3-42,2,39.9,39.9,0.0,39.9,Men's Fashion,cod,Valid,60124,2020,Nov-20,987867,Drs.,Jani,W,Titus,F,43,"Titus, Jani",jani.titus@gmail.com,8/22/2006,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
6,100367360,13/11/2020,order_refunded,595192,MATDAN59C3C845B38F0,2,47.6,47.6,0.0,47.6,Mobiles & Tablets,cod,Valid,60124,2020,Nov-20,987867,Drs.,Jani,W,Titus,F,43,"Titus, Jani",jani.titus@gmail.com,8/22/2006,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
7,100354677,01/10/2020,canceled,574769,GFE_19_USBLEDLight,2,49.0,49.0,0.0,49.0,Mobiles & Tablets,Payaxis,Gross,42485,2020,Oct-20,171143,Prof.,Lee,S,Eaker,M,28,"Eaker, Lee",lee.eaker@gmail.com,02/04/1981,239-335-6755,Graham,Bradford,Graham,FL,32042,South,lseaker,0.0
8,100354677,01/10/2020,canceled,574770,oasis_Kingston-32GB-DTIG4,2,135.0,135.0,0.0,135.0,Computing,Payaxis,Gross,42485,2020,Oct-20,171143,Prof.,Lee,S,Eaker,M,28,"Eaker, Lee",lee.eaker@gmail.com,02/04/1981,239-335-6755,Graham,Bradford,Graham,FL,32042,South,lseaker,0.0
9,100354677,01/10/2020,canceled,574771,Geepas_GSB5420,2,549.9,549.9,0.0,549.9,Appliances,Payaxis,Gross,42485,2020,Oct-20,171143,Prof.,Lee,S,Eaker,M,28,"Eaker, Lee",lee.eaker@gmail.com,02/04/1981,239-335-6755,Graham,Bradford,Graham,FL,32042,South,lseaker,0.0


In [5]:
def exploracion(df):
    df_info = pd.DataFrame()
    df_info["% nulos"] = round(df.isna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["% no_nulos"] = round(df.notna().sum()/df.shape[0]*100, 2).astype(str)+"%"
    df_info["tipo_dato"] = df.dtypes
    df_info["num_valores_unicos"] = df.nunique()
    print(f"""El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.
Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0], 2)}% de los datos.
Hay {len(list(df_info[(df_info["% nulos"] != "0.0%")].index))} columnas con datos nulos, y son:
{list(df_info[(df_info["% nulos"] != "0.0%")].index)}
y sin nulos hay {len(list(df_info[(df_info["% nulos"] == "0.0%")].index))} columnas y son:
{list(df_info[(df_info["% nulos"] == "0.0%")].index)}
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:""")
    display(df_info.head())
    print("Principales estadísticos de las columnas categóricas:")
    display(df.describe(include="O").T)
    print("Principales estadísticos de las columnas numéricas:")
    display(df.describe(exclude="O").T)
    return df_info

exploracion(df)

El DataFrame tiene 286392 filas y 35 columnas.
Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.
Hay 0 columnas con datos nulos, y son:
[]
y sin nulos hay 35 columnas y son:
['order_id', 'order_date', 'status', 'item_id', 'sku', 'qty_ordered', 'price', 'value', 'discount_amount', 'total', 'category', 'payment_method', 'bi_st', 'cust_id', 'year', 'month', 'ref_num', 'Name Prefix', 'First Name', 'Middle Initial', 'Last Name', 'Gender', 'age', 'full_name', 'E Mail', 'Sign in date', 'Phone No. ', 'Place Name', 'County', 'City', 'State', 'Zip', 'Region', 'User Name', 'Discount_Percent']
A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
order_id,0.0%,100.0%,object,201716
order_date,0.0%,100.0%,object,365
status,0.0%,100.0%,object,13
item_id,0.0%,100.0%,int64,286392
sku,0.0%,100.0%,object,47932


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
order_id,286392,201716,100476608,43
order_date,286392,365,20/12/2020,13678
status,286392,13,canceled,112166
sku,286392,47932,MATSAM59DB75ADB2F80,3775
category,286392,15,Mobiles & Tablets,61761
payment_method,286392,13,cod,102916
bi_st,286392,3,Gross,112333
month,286392,12,Dec-20,82528
Name Prefix,286392,7,Mr.,103506
First Name,286392,5161,Joel,2895


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
item_id,286392.0,741664.902131,95746.031827,574769.0,659684.50,742309.00,826124.25000,905208.00
qty_ordered,286392.0,3.011296,4.573837,1.0,2.00,2.00,3.00000,501.00
price,286392.0,851.385253,1741.750751,0.0,49.90,119.00,950.00000,101262.59
value,286392.0,885.879934,2073.252179,0.0,49.90,158.97,910.00000,101262.59
discount_amount,286392.0,70.039427,256.880870,0.0,0.00,0.00,18.38305,30213.15
total,286392.0,815.840507,1983.580302,0.0,49.90,149.80,800.00000,101262.59
cust_id,286392.0,70048.019054,30243.856395,4.0,56519.00,74225.50,92357.00000,115326.00
year,286392.0,2020.618778,0.485688,2020.0,2020.00,2021.00,2021.00000,2021.00
ref_num,286392.0,560853.663395,255828.418398,111127.0,341265.00,564857.00,781086.00000,999981.00
age,286392.0,46.489033,16.673288,18.0,32.00,47.00,61.00000,75.00


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
order_id,0.0%,100.0%,object,201716
order_date,0.0%,100.0%,object,365
status,0.0%,100.0%,object,13
item_id,0.0%,100.0%,int64,286392
sku,0.0%,100.0%,object,47932
qty_ordered,0.0%,100.0%,int64,72
price,0.0%,100.0%,float64,7561
value,0.0%,100.0%,float64,10607
discount_amount,0.0%,100.0%,float64,13732
total,0.0%,100.0%,float64,23755


# Valores unicos de las variables categoricas ✨

In [6]:
# creamos una lista con los nombres de las columnas categoricas 
columnas = df.select_dtypes(include='object').columns.tolist()
print(columnas)
# empezamos a iterar por cada una de las columnas para sacar sus valores únicos y sus frecuencias
for columna in columnas:
    print(f" \n----------- ESTAMOS ANALIZANDO LA COLUMNA: '{columna.upper()}' -----------\n")
    print(f"Sus valores únicos son: {df[columna].unique()}\n")
    print(f"Las frecuencias de los valores únicos de las categorías son: {df[columna].value_counts()} ")

['order_id', 'order_date', 'status', 'sku', 'category', 'payment_method', 'bi_st', 'month', 'Name Prefix', 'First Name', 'Middle Initial', 'Last Name', 'Gender', 'full_name', 'E Mail', 'Sign in date', 'Phone No. ', 'Place Name', 'County', 'City', 'State', 'Region', 'User Name']
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'ORDER_ID' -----------

Sus valores únicos son: [100354678 100354680 100367357 ... 100562383 100562384 100562386]

Las frecuencias de los valores únicos de las categorías son: order_id
100476608    43
100535753    41
100535749    40
100535744    40
100535739    40
             ..
100398603     1
100398610     1
100398613     1
100398619     1
100398593     1
Name: count, Length: 201716, dtype: int64 
 
----------- ESTAMOS ANALIZANDO LA COLUMNA: 'ORDER_DATE' -----------

Sus valores únicos son: ['01/10/2020' '13/11/2020' '08/10/2020' '21/10/2020' '24/12/2020'
 '28/12/2020' '22/10/2020' '24/10/2020' '25/10/2020' '09/10/2020'
 '16/10/2020' '17/10/2020' '19/10/2020' '02/1

# DUPLICADOS

In [7]:
def get_duplicate_rows(df):
    """
    Encuentra y devuelve las filas completas duplicadas de un DataFrame.

    Args:
        df (pd.DataFrame): El DataFrame de entrada.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las filas duplicadas completas.
    """
    # Identificar duplicados basados en todas las columnas
    duplicate_rows = df[df.duplicated(keep=False)]
    return duplicate_rows

get_duplicate_rows(df)

,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,category,payment_method,bi_st,cust_id,year,month,ref_num,Name Prefix,First Name,Middle Initial,Last Name,Gender,age,full_name,E Mail,Sign in date,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent
